In [1]:
from bs4 import BeautifulSoup
import csv
import re
import os 
import pandas as pd
import glob

In [ ]:
data = []

# reading html files from folder

#'Riv_Shelter_HTML_Files'is the name for the shelter HTML Files Riverside
#'Riv_Food_Pantry_HTML_Files'is the name for the food pantry HTML Files for Riverside 
#'Riv_Mental_Health_HTML_Files'is the name for the mental health HTML Files for Riverside
#'Downtown_LA_Food_Pantry_HTML_Files' is the name for the food pantry HTML Files for downtown LA
#'Irvine_Mental_Health_HTML_Files' is the name for the mental health HTML Files for Irvine
folder_path = 'Downtown_LA_Mental_Health_HTML_Files'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.html'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = BeautifulSoup(html_content, 'html.parser')


        # Find the <ul> element with class "best-programs"
        ul_element = soup.find('ul', {'class': 'best-programs'})

        # Find all <li> elements within the <ul>
        li_elements = ul_element.find_all('li', {'class': 'search-result card card-v3 program-info'})

        # Loop through the found <li> elements and extract labels and values
        for li_element in li_elements:
            # Use BeautifulSoup to parse the HTML of each li_element
            li_soup = BeautifulSoup(str(li_element), 'html.parser')

            # Find the program heading 
            program_heading_element = li_soup.find('div', {'class': 'card-heading'})
            Service_name = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'}).text
            print('Service name: ', Service_name)
    
            # Extract program URL
            program_url_element = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'})
            program_url = program_url_element['href'] if program_url_element else None
            Service_url = "https://www.auntbertha.com/"+program_url
            print('URL: ', Service_url)
            
            is_reviewed = False

            # Reviewed on
            Reviewd_on_element = program_heading_element.find('div', {'class': 'last-reviewed'})
            if Reviewd_on_element:  # Check if the element exists
                Reviewd_on_text = Reviewd_on_element.get_text(strip=True)  # Clean up the text
                match = re.search(r'\d{2}/\d{2}/\d{4}', Reviewd_on_text)
                Reviewd_on = match.group() if match else None
                if Reviewd_on:  # If we found a review date
                    is_reviewed = True  # Set the flag to True
                print("Reviewed on:", Reviewd_on)
            else:
                Reviewd_on = None  # Set to None if not found
                print("No review date found.")
    
            # Access class="program-tags"
            program_tags = li_soup.find('div', {'class': 'program-tags'})

            # Main Services
            main_s = []
            main_service_list = program_tags.find('ul', {'class': 'list-inline'})
            main_service_items = main_service_list.find_all('li')

            for item in main_service_items:
                main_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                main_s.append(main_service)

            print("Main Services:", main_s)

            # Try to access the "Other Services" section
            other_service_list = program_tags.find('div', {'class': 'secondary-tags'})
            if other_service_list:
                other_s = []
                other_service_items = other_service_list.find('ul', {'class': 'list-inline'}).find_all('li')

                for item in other_service_items:
                    other_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                    other_s.append(other_service)
            else:
                other_s = None

            print("Other Services:", other_s)

            # Access the "Serving" section
            serving_section = program_tags.find('div', {'class': 'attribute-tags'})

            # Initialize a list to store the serving information
            serving_ = []
    
            # Find all the <li> elements within the serving section
            serving_items = serving_section.find_all('li')

            # Iterate through the serving items and extract the text from the <a> elements
            for item in serving_items:
                link = item.find('a', {'class': 'loading-on-click activity-log'})
    
                # Check if the link was found
                if link:  # Only proceed if the link exists
                    serving_text = link.text.strip()
                    serving_.append(serving_text)

            print("Serving:", serving_)

    
            # accessing next-steps-module, extract phone number, location, hours 
    
            next_steps_module = li_soup.find('div', {'class': 'next-steps-module'})

            # Extract phone number
            phone_number_elements = next_steps_module.find_all('span', {'class': 'result-next-step-item'})

            # Also find all 'a' elements with href attributes containing 'tel:'
            tel_link_elements = next_steps_module.find_all('a', href=True)

            phone_number = None  # Initialize phone number as None

            # Extract phone numbers from text-based spans
            for element in phone_number_elements:
                phone_number_text = element.text.strip() if element else None
    
                # Regex to extract digits (handling separators like spaces or hyphens)
                phone_number_matches = re.findall(r'[\d-]+', phone_number_text)
    
                # Join the digits into a single phone number string if matches are found
                if phone_number_matches:
                    phone_number = ''.join(phone_number_matches)
                    break  # Stop once we've found a phone number

            # If no phone number found from text, check the 'tel:' href links
            if not phone_number:
                for element in tel_link_elements:
                    href_value = element['href']
        
                    # Check if the href contains 'tel:' and extract digits
                    if 'tel:' in href_value:
                        phone_number_matches = re.findall(r'[\d-]+', href_value)
                        if phone_number_matches:
                            phone_number = ''.join(phone_number_matches)
                            break  # Stop once we've found a phone number

            # Print the extracted phone number
            if phone_number:
                print("Phone Number:", phone_number)
            else:
                print("No phone number found.")

            # Extract location address
            location_address_element = next_steps_module.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click map-link with-address'})
            location_address = location_address_element.text.strip() if location_address_element else None
            location_address = re.sub(r'\s+', ' ', location_address) if location_address_element else None
            print("Location Address:", location_address)

            # Extract URL
            location_url_map = location_address_element['href'] if location_address_element else None
            print("Location url map:", location_url_map)
            
            hours_info = {
                '24_hour': False,
                'Monday': None,
                'Tuesday': None,
                'Wednesday': None,
                'Thursday': None,
                'Friday': None,
                'Saturday': None,
                'Sunday': None
            }
            
            # Extract hours based on structure
            hours_element = next_steps_module.find('div', {'class': 'office-hours-schedule see-hours-dropdown'})
            if hours_element:
                # Find all the <span> elements within the hours_element
                day_spans = hours_element.find_all('span')

                # Ensure that we have even pairs of day and corresponding hours
                if len(day_spans) % 2 == 0:
                    # Iterate through the spans in pairs (day and its corresponding hours)
                    for i in range(0, len(day_spans), 2):
                        day_span = day_spans[i]          # This is the day (e.g., "Monday:")
                        hours_span = day_spans[i + 1]    # This is the corresponding hours (e.g., "Closed" or time)

                        # Extract the day name and remove the colon
                        day = day_span.text.strip()[:-1]  # Remove the ':' at the end
                        full_day = day.strip()            # Ensure there's no extra space

                        # Check if the day is valid and exists in hours_info
                        if full_day in hours_info:
                            # Check if the corresponding hours span indicates "Closed"
                            if 'Closed' in hours_span.text:
                                hours_info[full_day] = 'Closed'
                            else:
                                # Extract hours for non-closed days
                                hours_info[full_day] = hours_span.text.strip()
                else:
                    print("Unexpected hours format. Ensure that each day has corresponding hours.")

            # Handle 24-hour information if it exists
            else:
                hours_element = next_steps_module.find('span', {'class': 'result-geo-hours'})
                if hours_element:
                    # Get 24-hour text strip
                    hours_text = hours_element.get_text(strip=True)

                    # Assuming the presence of this text means 24-hour operation
                    if '24' in hours_text or '24-hour' in hours_text.lower():
                        hours_info['24_hour'] = True
                    else:
                        print("Unable to determine if the office is 24 hours.")
                else:
                    print("Hours information not available.")

            # Output the result
            print(hours_info)
    
    
            # Loop through the found <li> elements and extract labels and values
            Extra_element = li_element.find('div', {'class': 'panel-wrapper more-info-panel'})
            elig = []
            eligibility_rules_element = Extra_element.find('div', {'class': 'eligibility-rules'})
    
            # Check if eligibility_rules_element is found
            if eligibility_rules_element:
                # Check if eligibility_rules_element contains a list (ul)
                ul_element = eligibility_rules_element.find('ul')
        
                if ul_element:
                    # If it contains a list, extract list items and store them in a flat list
                    eligibility_list = [li.text.strip() for li in ul_element.find_all('li')]
                    elig.extend(eligibility_list)  # Use extend to add elements to the list directly
                    #print("Eligibility:", eligibility_list)
                else:
                    # If it doesn't contain a list, store the text as is
                    eligibility_text = eligibility_rules_element.text.strip()
                    #print("Eligibility:", [eligibility_text])  # Wrap in a list to maintain consistency
                    elig.append(eligibility_text)

            else:
                # Skip if 'eligibility-rules' class is not found
                pass
    
        
            print("Eligibility:", elig)
    
            # Extract Availability
            availability_element = Extra_element.find('strong', {'data-translate': 'Availability'})
            
            if availability_element:
                availability = availability_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                availability = "Not specified"

            # Extract Description
            description_element = Extra_element.find('strong', {'data-translate': 'Description'})
            description = description_element.find_next('div', {'class': 'col-md-10'}).text.strip()

            # Extract Languages
            languages_element = Extra_element.find('strong', {'data-translate': 'Languages'})
            languages = languages_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            languages_data = [lang.strip() for lang in languages.split(',')]
            
            # Extract Cost
            cost_element = Extra_element.find('strong', string='Cost:')
            
            if cost_element:  # Check if cost_element exists
                cost = cost_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                cost = "Not specified"  # Default value if cost_element is not found
            
            # Extract Website URLs if they exist, or set them to None
            website_element = Extra_element.find('div', {'data-translation': 'Website'})
            website_url = website_element.find_next('a', {'class': 'activity-log descriptionProgramWebsite'})['href'] if website_element else None

            # Extract Facebook and Twitter URLs if they exist, or set them to None
            facebook_element = Extra_element.find('strong', {'data-translate': 'Facebook'})
            facebook_url = facebook_element.find_next('a', {'class': 'activity-log descriptionProgramFacebook'})['href'] if facebook_element else None

            twitter_element = Extra_element.find('strong', {'data-translate': 'Twitter'})
            twitter_url = twitter_element.find_next('a', {'class': 'activity-log descriptionProgramTwitter'})['href'] if twitter_element else None

            # Extract Coverage Area
            coverage_element = Extra_element.find('strong', {'data-translate': 'Coverage Area'})
            coverage = coverage_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            
            # Initialize latitude and longitude as None by default
            latitude = None
            longitude = None
            zipcode = None

            # Find the element with class "office-hour-address"
            location_element = Extra_element.find('div', {'class': 'office-hours-address _js_address address notranslate'})

            # Check if the element exists and has the required attributes
            if location_element:
                latitude = location_element['data-latitude'] if location_element.has_attr('data-latitude') else None
                longitude = location_element['data-longitude'] if location_element.has_attr('data-longitude') else None
                print("Latitude:", latitude)
                print("Longitude:", longitude)
                
                # Extract all text within location_element
                address_text = location_element.get_text(separator=" ").strip()  # Get all text as a single string

                # Use regular expression to search for the ZIP code pattern anywhere in the text
                zip_matches = re.findall(r'(?<!\d)(\b\d{5}\b)(?!\d)', address_text)
                zipcode = int(zip_matches[-1]) if zip_matches else None
                print("ZIP Code:", zipcode)

            # Print or use the extracted values as needed
            print("Availability:", availability)
            print("Description:", description)
            print("Languages:", languages_data)
            print("Cost:", cost)
            print("Facebook URL:", facebook_url)
            print("Twitter URL:", twitter_url)
            print("Coverage Area:", coverage)

            data.append([
            Service_name,
            Service_url,
            main_s,
            other_s,
            serving_,
            phone_number,
            website_url,
            location_address,
            location_url_map,
            elig,
            availability,
            description,
            languages_data,
            cost,
            is_reviewed,
            facebook_url,
            twitter_url,
            coverage,
            latitude,
            longitude,
            zipcode,
            hours_info['24_hour'],
            hours_info['Monday'],      
            hours_info['Tuesday'],     
            hours_info['Wednesday'],   
            hours_info['Thursday'],   
            hours_info['Friday'],     
            hours_info['Saturday'],  
            hours_info['Sunday']])    
            print("***************************************************************************")

Service name:  Women's Health
URL:  https://www.auntbertha.com//northeast-valley-health-corporation-%2528nevhc%2529--san-fernando-ca--women%2527s-health/5740419719757824?postal=90013
Reviewed on: 12/17/2024
Main Services: ['health education', 'family planning', 'checkup & test', 'disease screening', 'pregnancy tests', 'counseling', 'sexual and reproductive health', 'womens health', 'postnatal care', 'sex education', 'std/sti treatment & prevention', 'support network']
Other Services: None
Serving: ['adults', 'young adults', 'teens', 'pregnant', 'individuals', 'families', 'benefit recipients', 'low-income', 'uninsured']
Phone Number: 818-270-9777
Location Address: 1600 San Fernando Road, San Fernando, CA 91340
Location url map: https://www.google.com/maps/?q=1600+San+Fernando+Road,+San+Fernando,+CA+91340/
{'24_hour': False, 'Monday': '8:00 AM - 9:00 PM  PST', 'Tuesday': '8:00 AM - 9:00 PM  PST', 'Wednesday': '8:00 AM - 9:00 PM  PST', 'Thursday': '8:00 AM - 9:00 PM  PST', 'Friday': '9:00

Service name:  Medication Assisted Treatment (MAT)
URL:  https://www.auntbertha.com//western-pacific-med-corp--glendale-ca--medication-assisted-treatment-%2528mat%2529/6608374354411520?postal=90013
Reviewed on: 11/13/2024
Main Services: ['addiction & recovery', 'outpatient treatment', 'medications for addiction']
Other Services: None
Serving: ['adults 18+', 'individuals', 'benefit recipients', 'substance dependency', 'opioid dependency']
Phone Number: 818-956-3737
Location Address: 4544 San Fernando Road, Glendale, CA 91204
Location url map: https://www.google.com/maps/?q=4544+San+Fernando+Road,+Glendale,+CA+91204/
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program helps people who are older than 17 years old.', 'Must have Medi-Cal.']
Latitude: 34.1348928
Longitude:

Service name:  Intensive Clinical Program
URL:  https://www.auntbertha.com//home-base-program--boston-ma--intensive-clinical-program/6350991095103488?postal=90013
Reviewed on: 12/11/2024
Main Services: ['outpatient treatment', 'medication management', 'individual counseling', 'mental health evaluation']
Other Services: ['meals', 'transportation for healthcare', 'checkup & test', 'prevent & treat', 'medication management', 'mental health care', 'community support services', 'recreation', 'support network', 'support groups']
Serving: ['adults 18+', 'active duty', 'national guard', 'veterans', 'individuals', 'families', 'sexual assault survivors', 'brain injury', 'ptsd', 'all mental health']
Phone Number: 617-724-5202
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '9:00 AM - 5:00 PM', 'Tuesday': '9:00 AM - 5:00 PM', 'Wednesday': '9:00 AM - 5:00 PM', 'Thursday': '9:00 AM - 5:00 PM', 'Friday': '9:00 AM - 5:00 PM', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibi

Service name:  Mental Health Services
URL:  https://www.auntbertha.com//homeboy-industries--los-angeles-ca--mental-health-services/5071875430416384?postal=90013
Reviewed on: 12/14/2024
Main Services: ['addiction & recovery', 'health education', 'parenting education', 'specialized therapy', 'mental health care', 'counseling', 'substance abuse counseling', 'individual counseling', 'mental health evaluation', 'support network']
Other Services: None
Serving: ['adults', 'young adults', 'teens', 'seniors', 'criminal justice history', 'trauma survivors', 'domestic violence survivors', 'all mental health', 'substance dependency']
Phone Number: 323-526-1254
Location Address: 130 Bruno Street, Los Angeles, CA 90012
Location url map: https://www.google.com/maps/?q=130+Bruno+Street,+Los+Angeles,+CA+90012/
{'24_hour': False, 'Monday': '9:00 AM - 5:00 PM  PST', 'Tuesday': '9:00 AM - 5:00 PM  PST', 'Wednesday': '9:00 AM - 5:00 PM  PST', 'Thursday': '9:00 AM - 5:00 PM  PST', 'Friday': '9:00 AM - 5:00 

Service name:  Coaching Into Care
URL:  https://www.auntbertha.com//us-department-of-veterans-affairs-%2528va%2529---veterans-health-administration-%2528vha%2529--washington-dc--coaching-into-care/5427922043666432?postal=90013
Reviewed on: 12/15/2024
Main Services: ['mental health care', 'understand mental health', 'government benefits', 'understand government programs', 'navigating the system', 'support network', 'one-on-one support']
Other Services: None
Serving: ['adults', 'young adults', 'seniors', 'national guard', 'veterans', 'families', 'caregivers', 'spouses', 'parents', 'depression', 'brain injury', 'anxiety', 'ptsd', 'all mental health', 'substance dependency']
Phone Number: 888-823-7458
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '5:00 AM - 5:00 PM  PST', 'Tuesday': '5:00 AM - 5:00 PM  PST', 'Wednesday': '5:00 AM - 5:00 PM  PST', 'Thursday': '5:00 AM - 5:00 PM  PST', 'Friday': '5:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}


...

Service name:  Therapeutic Behavioral Service (TBS)
URL:  https://www.auntbertha.com//childnet-youth-and-family-services--long-beach-ca--therapeutic-behavioral-service-%2528tbs%2529/4801969775706112?postal=90013
Reviewed on: 12/13/2024
Main Services: ['daily life skills', 'parenting education', 'mental health care', 'psychiatric emergency services', 'adoption & foster care', 'post-adoption support', 'one-on-one support']
Other Services: None
Serving: ['teens', 'children', 'foster youth', 'families', 'single parent', 'caregivers', 'parents', 'in crisis', 'all mental health']
Phone Number: 562-490-7600
Location Address: 4155 Outer Traffic Circle, Long Beach, CA 90804
Location url map: https://www.google.com/maps/?q=4155+Outer+Traffic+Circle,+Long+Beach,+CA+90804/
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', '

Eligibility: []
Latitude: 33.7802724
Longitude: -118.185324
ZIP Code: 90813
Availability: available
Description: LBTRC provides evidence-based trauma treatment for victims of crimes and their families. Our services provided a better quality by providing mental health care, outreach and education.Services provided:- Trauma treatment and case management for victims and their families - Individual Therapy Sessions- Group Therapy Sessions- Couple's Therapy (Trauma Related)- Psychiatric Referral and Medication Follow-up- Walk in Consults - Case ManagementWe also provide individualized assistance to linkages for:- Physical health issues- Housing/relocation/school services- Legal issues- Food- Employment- Emergency AssistanceServices at the LBTRC are provided at free of charge.Services at the LBTRC are provided by clinicians and advanced graduate students who are under the direct supervision of faculty in the College of Education. Clients are accepted on a first come, first served basis and o

Service name:  Anger Management Group
URL:  https://www.auntbertha.com//valley-women%2527s-center-%2528vwc%2529--los-angeles-ca--anger-management-group/4831429947031552?postal=90013
Reviewed on: 12/13/2024
Main Services: ['specialized therapy', 'anger management', 'skills & training']
Other Services: None
Serving: ['adults', 'young adults', 'teens', 'seniors', 'benefit recipients', 'low-income']
Phone Number: 818-713-8700
Location Address: 22110 Roscoe Boulevard, Los Angeles, CA 91304
Location url map: https://www.google.com/maps/?q=22110+Roscoe+Boulevard,+Los+Angeles,+CA+91304/
{'24_hour': False, 'Monday': '2:00 PM - 5:30 PM  PST', 'Tuesday': 'Closed', 'Wednesday': 'Closed', 'Thursday': 'Closed', 'Friday': 'Closed', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['Must have Medi-Cal.']
Latitude: 34.2193483
Longitude: -118.6083507
ZIP Code: 91304
Availability: available
Description: Valley Women's Center offers this group for those with anger issues. The group focuses on the ma

Service name:  Peer Mentor Program
URL:  https://www.auntbertha.com//tri-city-mental-health--pomona-ca--peer-mentor-program/4743685342232576?postal=90013
Reviewed on: 12/13/2024
Main Services: ['counseling', 'navigating the system', 'peer support']
Other Services: None
Serving: ['young adults', 'teens', 'seniors']
Phone Number: 909-623-6131
Location Address: 1403 N Garey Ave, Pomona, CA 91767
Location url map: https://www.google.com/maps/?q=1403+N+Garey+Ave,+Pomona,+CA+91767/
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: []
Latitude: 34.0717736
Longitude: -117.7526603
ZIP Code: 91767
Availability: available
Description: the Peer Mentor Program is available to persons of all ages within the Tri-City service area.  Volunteer mentors receive specialized training and on-going wee

Service name:  Helping Military Children Handle Loss Program
URL:  https://www.auntbertha.com//the-comfort-crew-for-military-kids--austin-tx--helping-military-children-handle-loss-program/5764545082818560?postal=90013
Reviewed on: 12/17/2024
Main Services: ['support network', 'bereavement']
Other Services: None
Serving: ['teens', 'children', 'veterans', 'grieving', 'families', 'spouses', 'dependents']
No phone number found.
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '6:00 AM - 3:00 PM  PST', 'Tuesday': '6:00 AM - 3:00 PM  PST', 'Wednesday': '6:00 AM - 3:00 PM  PST', 'Thursday': '6:00 AM - 3:00 PM  PST', 'Friday': '6:00 AM - 3:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program helps surviving children six and up of deceased military service members.']
Availability: waitlist
Description: The Helping Military Children Handle Loss Program gives children ages six and up the tools and strategies to understand the emotions that t

In [ ]:
# Save to CSV file 
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv for riv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv for riv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv for riv
#"FindHelp_extracted_data_downtown_la_food_pantry.csv" is the name for the food pantry csv for downtown la
csv_filename = "FindHelp_extracted_data_downtown_la_mental_health.csv"

with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # The header 
    header = [
        "Service_name",
        "Service_url",
        "Main_Services",
        "Other_Services",
        "Serving",
        "Phone_Number",
        "Website",
        "Location_Address",
        "Location_URL_Map",
        "Eligibility",
        "Availability",
        "Description",
        "Languages",
        "Cost",
        "Google_Review",
        "Facebook_URL",
        "Twitter_URL",
        "Coverage",
        "Latitude",
        "Longitude",
        "Zipcode",
        "24hour",
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday"
    ]
    csv_writer.writerow(header)
    
    csv_writer.writerows(data)

In [ ]:
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_downtown_la_food_pantry.csv"
#"FindHelp_extracted_data_downtown_la_shelter.csv"
#"FindHelp_extracted_data_downtown_la_mental_health.csv"
#"FindHelp_extracted_data_irvine_food_pantry.csv"
#"FindHelp_extracted_data_irvine_shelter.csv"
#"FindHelp_extracted_data_irvine_mental_health.csv"
df = pd.read_csv("FindHelp_extracted_data_downtown_la_mental_health.csv")

#riv_zipcodes = [92501, 92502, 92503, 92504, 92505, 92506, 92507, 92508, 92513, 92514, 92516, 92517, 92521, 92522]

downtown_la_zipcodes = [90012, 90013, 90014, 90015, 90017, 90021]

#irvine_zipcodes = [92602, 92603, 92604, 92606, 92612, 92614, 92617, 92618, 92620]

filtered_df = df[df['Zipcode'].isin(downtown_la_zipcodes) | (df['Location_Address'].isna() & df['Longitude'].isna() & df['Latitude'].isna())]

filtered_df = filtered_df.drop_duplicates(subset = ['Website', 'Service_name'])

filtered_df.to_csv("FindHelp_extracted_data_downtown_la_mental_health.csv", index=False, encoding='utf-8')

filtered_df